# Please do not re-run this file on deepnote.

In [ ]:
# Uncomment & Run these codes if you don't have them yet
#!pip install librosa
#!pip install --user --upgrade librosa
#!pip install --user --upgrade numba

In [ ]:
# just in case
#for file in os.listdir():
    #if file != '.deepnote' and file != 'data_path.csv':
        #print(file)
        #os.remove(file)

In [1]:
import pandas as pd
import numpy as np
import os

import librosa as l
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder 

In [2]:
data_path = pd.read_csv('data_path.csv')
data_path.head()

,Emotions,Path
0,male_neutral,Ravdess\03-01-01-01-01-01-01.wav
1,female_neutral,Ravdess\03-01-01-01-01-01-02.wav
2,male_neutral,Ravdess\03-01-01-01-01-01-03.wav
3,female_neutral,Ravdess\03-01-01-01-01-01-04.wav
4,male_neutral,Ravdess\03-01-01-01-01-01-05.wav


# 1. Data Pre-Processing

# Feature Extraction from the Audio files

- Tone Colour --> MFCC (librosa):
The mel frequency cepstral coefficients (MFCCs) of an audio signal are a small set of features (usually about 10–20) which describe the overall shape of the spectral envelope. It was also used to describe “timbre”, that unique “something” that gives color and personality to your voice, and how it is recognized.

- Energy of Pitch --> Chroma (librosa)
Chroma feature represents the energy distribution of pitch classes, which are the 12 different pitch classes in the chromatic scale, representing the tonal content of an audio signal. In speech emotion recognition, chroma features may capture variations in pitch that convey emotional information.

- Frequency --> MEL Spectrogram Frequency (librosa)
Mel spectrogram represents the power spectrum of a signal, where the frequencies are converted to the mel scale, which is more perceptually relevant that approximates the human ear's sensitivity to different frequencies. In speech emotion recognition, they can provide information about the distribution of energy across different frequency bands, which may be relevant for emotional content.

- Contrast in pitch --> Spectral Contrast (librosa)
Spectral contrast measures the difference in amplitude between peaks and valleys in the spectrum, providing information about the overall tonal characteristics. Contrast can be useful in capturing variations in spectral content. In the context of speech emotion recognition, it may capture variations in the pitch or emphasis of certain words or phonemes. 

- Changes/Shift in Tone --> Tonnetz (librosa)
Tonnetz features capture tonal centroid features representing tonal content in the audio signal. Tonnetz is sensitive to harmonic relationships and can be helpful in differentiating between different tonal qualities in speech. In the context of speech emotion recognition, changes in tonal qualities, such as shifts in pitch or intonation, can carry emotional information.

In [3]:
# Classifying which folder each row belongs to
data_path

,Emotions,Path
0,male_neutral,Ravdess\03-01-01-01-01-01-01.wav
1,female_neutral,Ravdess\03-01-01-01-01-01-02.wav
2,male_neutral,Ravdess\03-01-01-01-01-01-03.wav
3,female_neutral,Ravdess\03-01-01-01-01-01-04.wav
4,male_neutral,Ravdess\03-01-01-01-01-01-05.wav
...,...,...
11677,female_sad,Tess\YAF_sad\YAF_witch_sad.wav
11678,female_sad,Tess\YAF_sad\YAF_yearn_sad.wav
11679,female_sad,Tess\YAF_sad\YAF_yes_sad.wav
11680,female_sad,Tess\YAF_sad\YAF_young_sad.wav


In [4]:
# Audio Files Location
#crema_path = "C:/Users/Rachel/Desktop/My Projects/Speech Emotion Recognition/wav_file/Crema"
#ravdess_path = "C:/Users/Rachel/Desktop/My Projects/Speech Emotion Recognition/wav_file/Ravdess"
#tess_path = "C:/Users/Rachel/Desktop/My Projects/Speech Emotion Recognition/wav_file/Tess"

common_path = "C:\\Users\\Rachel\\Desktop\\My Projects\\Speech Emotion Recognition\\wav_file\\" # to change to YOUR own

In [9]:
def feature_extraction_mfcc(data, sample_rate):
    # Extract features from the audio
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    return mfcc

def feature_extraction_sc(data, sample_rate):
    # Extract features from the audio
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=data, sr=sample_rate).T,axis=0)
    return spectral_contrast

def feature_extraction_chroma(data, sample_rate):
    # Extract features from the audio
    chroma = np.mean(librosa.feature.chroma_stft(y=data, sr=sample_rate).T,axis=0)
    return chroma

def feature_extraction_mel(data, sample_rate):
    # Extract features from the audio
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T,axis=0)
    return mel

def feature_extraction_tonnetz(data, sample_rate):
    # Extract features from the audio
    tonnetz = np.mean(librosa.feature.tonnetz(y=data, sr=sample_rate).T,axis=0)
    return tonnetz

In [10]:
# looping through ALL wav files --> takes about 20-30 minutes to run
results = []
print(common_path + data_path.Path[0])

for audio in data_path.Path:
    initial = {}
    
    directory = common_path + audio
    #print(directory)
    data, sample_rate = librosa.load(directory)
    
    feature_mfcc = feature_extraction_mfcc(data, sample_rate)  
    initial['MFCC'] = feature_mfcc
    
    feature_sc = feature_extraction_sc(data, sample_rate)  
    initial['Spectral Contrast'] = feature_sc
    
    feature_c = feature_extraction_chroma(data, sample_rate)  
    initial['Chroma'] = feature_c
    
    feature_mel = feature_extraction_mel(data, sample_rate)  
    initial['Mel Spectogram'] = feature_mel
    
    feature_tonz = feature_extraction_tonnetz(data, sample_rate)  
    initial['Tonnetz'] = feature_tonz
    
    results.append(initial)
    
results

C:\Users\Rachel\Desktop\My Projects\Speech Emotion Recognition\wav_file\Ravdess\03-01-01-01-01-01-01.wav


C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=1012
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=785
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=805
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=748
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=644
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=702
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\l

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=824
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=531
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=557
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=532
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=554
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\li

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=996
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=498
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=1015
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=508
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=537
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=595
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\l

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=1006
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=503
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=950
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=548
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=538
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=565
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\l

...

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=700
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=676
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=688
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=651
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=617
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=664
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\li

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=639
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=772
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=740
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=714
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=733
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=710
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\li

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=871
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=921
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=906
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=980
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=843
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=945
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\li

C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=994
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=490
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=602
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=625
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=606
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=604
  warnings.warn(
C:\Users\Rachel\anaconda3\lib\site-packages\li

[{'MFCC': array([-6.9779260e+02,  5.4890041e+01,  6.6346556e-01,  1.2435786e+01,
          7.7339516e+00,  5.3075039e-01, -3.2166309e+00, -3.1593943e+00,
         -1.0977551e+01, -2.8487110e+00,  8.1529748e-01, -3.0370669e+00,
          1.9554467e+00, -3.7356679e+00,  4.3770033e-01,  7.8235668e-01,
         -4.0714288e+00, -1.8607764e+00, -1.3236899e+00, -9.2046332e-01],
        dtype=float32),
  'Spectral Contrast': array([19.06341554, 11.11139791, 14.83170261, 14.06422013, 14.78303579,
         16.76310353, 45.3462458 ]),
  'Chroma': array([0.6550447 , 0.61074096, 0.56857884, 0.57231134, 0.5553723 ,
         0.52929276, 0.5870647 , 0.638355  , 0.6429177 , 0.6064223 ,
         0.6101092 , 0.60403866], dtype=float32),
  'Mel Spectogram': array([1.3577937e-06, 5.5969314e-05, 3.6217349e-03, 2.0932278e-02,
         3.1378519e-02, 1.7672727e-02, 3.4504111e-03, 1.8597402e-02,
         2.4657493e-02, 2.0443266e-02, 2.1946169e-02, 1.0737971e-02,
         7.9423701e-03, 7.2973408e-03, 1.509808

In [ ]:
#!pip install numba

In [13]:
results_temp = results.copy() # original results df - go back to this 

# concatenating data_path df & results_df derived from above
results_df = pd.DataFrame(results_temp)
results_df

temp_data_path = data_path.copy()
temp_data_path

df_concat = pd.concat([temp_data_path,results_df], axis=1)
df_concat

,Emotions,Path,MFCC,Spectral Contrast,Chroma,Mel Spectogram,Tonnetz
0,male_neutral,Ravdess\03-01-01-01-01-01-01.wav,"[-697.7926, 54.89004, 0.66346556, 12.435786, 7...","[19.063415539318775, 11.111397906331113, 14.83...","[0.6550447, 0.61074096, 0.56857884, 0.57231134...","[1.3577937e-06, 5.5969314e-05, 0.003621735, 0....","[-0.049044148082291565, 0.020023551860533305, ..."
1,female_neutral,Ravdess\03-01-01-01-01-01-02.wav,"[-650.7109, 54.477303, -9.090127, 8.411754, -4...","[17.29156506391496, 15.724447099708668, 16.684...","[0.36715057, 0.36417577, 0.34141716, 0.3937550...","[1.2319014e-05, 5.216602e-05, 0.000112376976, ...","[-0.019343583888426674, 0.01213930326473252, 0..."
2,male_neutral,Ravdess\03-01-01-01-01-01-03.wav,"[-614.73914, 56.70819, -2.6855354, 10.650176, ...","[20.576568236634287, 12.587620530770842, 16.07...","[0.4724504, 0.45587808, 0.4547249, 0.47586885,...","[2.669849e-05, 0.00019187588, 0.00078644586, 0...","[-0.013745768690970545, -0.006524704891460672,..."
3,female_neutral,Ravdess\03-01-01-01-01-01-04.wav,"[-695.8503, 42.934265, -7.2745566, 8.977729, -...","[17.095491688835857, 15.326973089860852, 15.85...","[0.38081443, 0.35138282, 0.37724286, 0.4280217...","[6.649222e-06, 9.887024e-06, 1.460355e-05, 7.5...","[-0.00660110324909748, 0.012612832449873611, -..."
4,male_neutral,Ravdess\03-01-01-01-01-01-05.wav,"[-713.4335, 68.36094, 7.989171, 15.139791, 11....","[19.039203251129084, 12.024992571333968, 15.29...","[0.51001614, 0.58901775, 0.60913634, 0.5573833...","[0.00011854509, 0.00011694191, 0.0011355863, 0...","[-0.023409436975484803, 0.016631760474731867, ..."
...,...,...,...,...,...,...,...
11677,female_sad,Tess\YAF_sad\YAF_witch_sad.wav,"[-391.5477, 61.473328, 34.77998, 42.554768, -2...","[16.24730919658086, 19.9651103386942, 21.52398...","[0.41963473, 0.37687376, 0.3121896, 0.22717477...","[0.17052567, 0.13962038, 0.0825388, 0.05848019...","[-0.04658024786744, -0.005804598174103059, -0...."
11678,female_sad,Tess\YAF_sad\YAF_yearn_sad.wav,"[-404.34048, 76.25154, 25.959827, 39.57593, 0....","[22.096955661018278, 22.51148845550154, 23.683...","[0.20017977, 0.34767687, 0.2420479, 0.14724383...","[0.02069263, 0.01239855, 0.009333166, 0.008634...","[-0.043205504141752414, 0.09204317799819632, -..."
11679,female_sad,Tess\YAF_sad\YAF_yes_sad.wav,"[-370.4846, 65.6685, 38.36494, 41.20847, -4.55...","[16.369624156977487, 20.15629752235118, 22.252...","[0.37956417, 0.45231053, 0.303792, 0.22826175,...","[0.09469574, 0.042661775, 0.030507604, 0.03933...","[-0.05519286804179551, -0.001120963115530329, ..."
11680,female_sad,Tess\YAF_sad\YAF_young_sad.wav,"[-423.07797, 69.03684, 29.720037, 37.226368, -...","[18.47478665056106, 22.379059455979373, 23.279...","[0.33586672, 0.25874785, 0.19461411, 0.1549673...","[0.13651857, 0.2267648, 0.108386785, 0.0299305...","[-0.05507254009730232, -0.026233353679316736, ..."


In [38]:
df_merged_v2 = df_concat.copy()

# splitting array of each variable into separate columns & renaming it for easy reference
mfcc_values = pd.DataFrame(df_merged_v2['MFCC'].tolist())
num_of_cols = len(mfcc_values.columns)
new_columns_names = [f"MFCC{i}" for i in range(num_of_cols)] # Generate new unique column names
mfcc_values.columns = new_columns_names
mfcc_values

sc_values = pd.DataFrame(df_merged_v2['Spectral Contrast'].tolist())
num_of_cols = len(sc_values.columns)
new_columns_names = [f"SC{i}" for i in range(num_of_cols)] # Generate new unique column names
sc_values.columns = new_columns_names
sc_values

c_values = pd.DataFrame(df_merged_v2['Chroma'].tolist())
num_of_cols = len(c_values.columns)
new_columns_names = [f"C{i}" for i in range(num_of_cols)] # Generate new unique column names
c_values.columns = new_columns_names
c_values

mel_values = pd.DataFrame(df_merged_v2['Mel Spectogram'].tolist())
num_of_cols = len(mel_values.columns)
new_columns_names = [f"MEL{i}" for i in range(num_of_cols)] # Generate new unique column names
mel_values.columns = new_columns_names
mel_values

tonz_values = pd.DataFrame(df_merged_v2['Tonnetz'].tolist())
num_of_cols = len(tonz_values.columns)
new_columns_names = [f"TONZ{i}" for i in range(num_of_cols)] # Generate new unique column names
tonz_values.columns = new_columns_names
tonz_values

# Concatenate all dataframes ^ 
values_df = pd.concat([mfcc_values,sc_values,c_values,mel_values,tonz_values], axis=1)
values_df

,MFCC0,MFCC1,MFCC2,MFCC3,MFCC4,MFCC5,MFCC6,MFCC7,MFCC8,MFCC9,...,MEL124,MEL125,MEL126,MEL127,TONZ0,TONZ1,TONZ2,TONZ3,TONZ4,TONZ5
0,-697.792603,54.890041,0.663466,12.435786,7.733952,0.530750,-3.216631,-3.159394,-10.977551,-2.848711,...,0.000006,0.000005,0.000004,3.206722e-07,-0.049044,0.020024,-0.018065,-0.064224,0.014611,0.006371
1,-650.710876,54.477303,-9.090127,8.411754,-4.387536,-3.765706,-5.362752,-8.610381,-3.971552,-0.895974,...,0.000006,0.000006,0.000004,3.992178e-07,-0.019344,0.012139,0.013491,-0.040532,0.006054,0.002813
2,-614.739136,56.708191,-2.685535,10.650176,4.240806,-2.472097,-12.035720,-6.654510,-3.152715,-8.526167,...,0.000036,0.000071,0.000045,4.472179e-06,-0.013746,-0.006525,0.013147,-0.001333,0.005258,-0.001753
3,-695.850281,42.934265,-7.274557,8.977729,-4.170578,-4.924890,-6.537960,-12.679187,-7.311471,-3.600855,...,0.000017,0.000013,0.000006,6.443871e-07,-0.006601,0.012613,-0.023542,0.016175,-0.010311,0.000834
4,-713.433472,68.360939,7.989171,15.139791,11.715775,0.430983,1.002558,-2.773119,-0.127347,1.893249,...,0.000002,0.000002,0.000001,7.229193e-08,-0.023409,0.016632,-0.042659,0.019653,0.014472,0.010889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,-391.547699,61.473328,34.779980,42.554768,-2.182076,8.699821,-1.409615,-11.303340,0.550166,-4.555710,...,0.003400,0.006326,0.003662,3.548996e-04,-0.046580,-0.005805,-0.067837,0.051074,-0.006935,0.019644
11678,-404.340485,76.251541,25.959827,39.575932,0.379419,-0.940992,-6.228340,-14.371888,1.734153,-11.790402,...,0.006170,0.003555,0.001505,1.386760e-04,-0.043206,0.092043,-0.105126,0.011605,0.007710,-0.057523
11679,-370.484589,65.668503,38.364941,41.208469,-4.550175,4.382659,-0.712863,-9.426854,2.177757,-7.615240,...,0.006319,0.003989,0.001395,1.162493e-04,-0.055193,-0.001121,-0.075881,0.144801,-0.018379,-0.001876
11680,-423.077972,69.036842,29.720037,37.226368,-3.204174,13.681798,-3.380287,-12.161002,-0.653988,-4.454652,...,0.002952,0.003282,0.001737,8.170844e-05,-0.055073,-0.026233,-0.041737,0.033254,-0.058102,0.020330


In [39]:
# Concatenating df_concat & values_df
final_df = pd.concat([df_concat,values_df], axis=1)
final_df = final_df.drop(columns=['MFCC','Spectral Contrast','Chroma','Mel Spectogram','Tonnetz'])
final_df

# Separating emotions from gender
emotions_split = final_df['Emotions'].str.split("_", expand=True)
emotions = emotions_split[1]
final_df['Emotions_without_gender'] = emotions

final_df

,Emotions,Path,MFCC0,MFCC1,MFCC2,MFCC3,MFCC4,MFCC5,MFCC6,MFCC7,...,MEL125,MEL126,MEL127,TONZ0,TONZ1,TONZ2,TONZ3,TONZ4,TONZ5,Emotions_without_gender
0,male_neutral,Ravdess\03-01-01-01-01-01-01.wav,-697.792603,54.890041,0.663466,12.435786,7.733952,0.530750,-3.216631,-3.159394,...,0.000005,0.000004,3.206722e-07,-0.049044,0.020024,-0.018065,-0.064224,0.014611,0.006371,neutral
1,female_neutral,Ravdess\03-01-01-01-01-01-02.wav,-650.710876,54.477303,-9.090127,8.411754,-4.387536,-3.765706,-5.362752,-8.610381,...,0.000006,0.000004,3.992178e-07,-0.019344,0.012139,0.013491,-0.040532,0.006054,0.002813,neutral
2,male_neutral,Ravdess\03-01-01-01-01-01-03.wav,-614.739136,56.708191,-2.685535,10.650176,4.240806,-2.472097,-12.035720,-6.654510,...,0.000071,0.000045,4.472179e-06,-0.013746,-0.006525,0.013147,-0.001333,0.005258,-0.001753,neutral
3,female_neutral,Ravdess\03-01-01-01-01-01-04.wav,-695.850281,42.934265,-7.274557,8.977729,-4.170578,-4.924890,-6.537960,-12.679187,...,0.000013,0.000006,6.443871e-07,-0.006601,0.012613,-0.023542,0.016175,-0.010311,0.000834,neutral
4,male_neutral,Ravdess\03-01-01-01-01-01-05.wav,-713.433472,68.360939,7.989171,15.139791,11.715775,0.430983,1.002558,-2.773119,...,0.000002,0.000001,7.229193e-08,-0.023409,0.016632,-0.042659,0.019653,0.014472,0.010889,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,female_sad,Tess\YAF_sad\YAF_witch_sad.wav,-391.547699,61.473328,34.779980,42.554768,-2.182076,8.699821,-1.409615,-11.303340,...,0.006326,0.003662,3.548996e-04,-0.046580,-0.005805,-0.067837,0.051074,-0.006935,0.019644,sad
11678,female_sad,Tess\YAF_sad\YAF_yearn_sad.wav,-404.340485,76.251541,25.959827,39.575932,0.379419,-0.940992,-6.228340,-14.371888,...,0.003555,0.001505,1.386760e-04,-0.043206,0.092043,-0.105126,0.011605,0.007710,-0.057523,sad
11679,female_sad,Tess\YAF_sad\YAF_yes_sad.wav,-370.484589,65.668503,38.364941,41.208469,-4.550175,4.382659,-0.712863,-9.426854,...,0.003989,0.001395,1.162493e-04,-0.055193,-0.001121,-0.075881,0.144801,-0.018379,-0.001876,sad
11680,female_sad,Tess\YAF_sad\YAF_young_sad.wav,-423.077972,69.036842,29.720037,37.226368,-3.204174,13.681798,-3.380287,-12.161002,...,0.003282,0.001737,8.170844e-05,-0.055073,-0.026233,-0.041737,0.033254,-0.058102,0.020330,sad


In [41]:
# label encoding emotions
label_encoder = LabelEncoder()
final_df['Emotions_without_gender'] = label_encoder.fit_transform(final_df['Emotions_without_gender'])
# 0 -> Angry
# 1 -> Calm
# 2 -> Disgust
# 3 -> Fear
# 4 -> Happy
# 5 -> Neutral
# 6 -> Sad
# 7 -> Surprised

# Rename Emotions Column and Label Encoding 
final_df.rename(columns = {'Emotions':'Emotions_with_gender'}, inplace = True)
label_encoder = LabelEncoder()
final_df['Emotions_with_gender'] = label_encoder.fit_transform(final_df['Emotions_with_gender'])
#  0 -> female_angry
#  1 -> female_calm
#  2 -> female_disgust
#  3 -> female_fear
#  4 -> female_happy
#  5 -> female_neutral
#  6 -> female_sad
#  7 -> female_surprise
#  8 -> male_angry
#  9 -> male_calm
# 10 -> male_disgust
# 11 -> male_fear
# 12 -> male_happy
# 13 -> male_neutral
# 14 -> male_sad
# 15 -> male_surprise

final_df

,Emotions_with_gender,Path,MFCC0,MFCC1,MFCC2,MFCC3,MFCC4,MFCC5,MFCC6,MFCC7,...,MEL125,MEL126,MEL127,TONZ0,TONZ1,TONZ2,TONZ3,TONZ4,TONZ5,Emotions_without_gender
0,13,Ravdess\03-01-01-01-01-01-01.wav,-697.792603,54.890041,0.663466,12.435786,7.733952,0.530750,-3.216631,-3.159394,...,0.000005,0.000004,3.206722e-07,-0.049044,0.020024,-0.018065,-0.064224,0.014611,0.006371,5
1,5,Ravdess\03-01-01-01-01-01-02.wav,-650.710876,54.477303,-9.090127,8.411754,-4.387536,-3.765706,-5.362752,-8.610381,...,0.000006,0.000004,3.992178e-07,-0.019344,0.012139,0.013491,-0.040532,0.006054,0.002813,5
2,13,Ravdess\03-01-01-01-01-01-03.wav,-614.739136,56.708191,-2.685535,10.650176,4.240806,-2.472097,-12.035720,-6.654510,...,0.000071,0.000045,4.472179e-06,-0.013746,-0.006525,0.013147,-0.001333,0.005258,-0.001753,5
3,5,Ravdess\03-01-01-01-01-01-04.wav,-695.850281,42.934265,-7.274557,8.977729,-4.170578,-4.924890,-6.537960,-12.679187,...,0.000013,0.000006,6.443871e-07,-0.006601,0.012613,-0.023542,0.016175,-0.010311,0.000834,5
4,13,Ravdess\03-01-01-01-01-01-05.wav,-713.433472,68.360939,7.989171,15.139791,11.715775,0.430983,1.002558,-2.773119,...,0.000002,0.000001,7.229193e-08,-0.023409,0.016632,-0.042659,0.019653,0.014472,0.010889,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,6,Tess\YAF_sad\YAF_witch_sad.wav,-391.547699,61.473328,34.779980,42.554768,-2.182076,8.699821,-1.409615,-11.303340,...,0.006326,0.003662,3.548996e-04,-0.046580,-0.005805,-0.067837,0.051074,-0.006935,0.019644,6
11678,6,Tess\YAF_sad\YAF_yearn_sad.wav,-404.340485,76.251541,25.959827,39.575932,0.379419,-0.940992,-6.228340,-14.371888,...,0.003555,0.001505,1.386760e-04,-0.043206,0.092043,-0.105126,0.011605,0.007710,-0.057523,6
11679,6,Tess\YAF_sad\YAF_yes_sad.wav,-370.484589,65.668503,38.364941,41.208469,-4.550175,4.382659,-0.712863,-9.426854,...,0.003989,0.001395,1.162493e-04,-0.055193,-0.001121,-0.075881,0.144801,-0.018379,-0.001876,6
11680,6,Tess\YAF_sad\YAF_young_sad.wav,-423.077972,69.036842,29.720037,37.226368,-3.204174,13.681798,-3.380287,-12.161002,...,0.003282,0.001737,8.170844e-05,-0.055073,-0.026233,-0.041737,0.033254,-0.058102,0.020330,6


In [42]:
# convert to csv file
final_df.to_csv('final_df.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0b28ef1e-f6a6-4523-8903-70adcffed1c5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>